Below you will find the routine to compute the ensemble mean position. It is in FORTRAN but I think you can understand the algorithm (let me know if you still have troubles to translate). Basically the inputs are knpf (dimensions of the vector of rlatpf and rlonpf), rlatpf & rlonpf the vectors containing the latitudes and longitudes of the ensemble for one time  step ( this routine is called for each forecast time step!). The output are the pair rlatmean and rlonmean (average position for that time step). Let me know if you have any other questions!

knps -> dimensions of the vector of rlatpf and rlonpf \
rlatpf, rlonpf -> vectors containing the latitudes and longitudes of the ensemble for one time step \
rlatmean, rlonmean -> output average position for that time step

In [59]:
import pdbufr
import math
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [60]:
def meanposit(knpf, rlatpf, rlonpf):
    rpi = math.acos(0.0)
    rnomin = 0.0
    rdenom = 0.0
    rphisum = 0.0
    
    for k in range(knpf):
        rlat = rlatpf[k] * rpi / 180.0
        rlon = rlonpf[k] * rpi / 180.0
        rcosphi = math.cos(rlat)
        rnomin += rcosphi * rlon
        rdenom += rcosphi
        rphisum += rlat
    
    rlabda = rnomin / rdenom
    rlonmean = rlabda * 180.0 / rpi
    
    rnomin = 0.0
    repsilon = 0.0
    
    for k in range(knpf):
        rlat = rlatpf[k] * rpi / 180.0
        rlon = rlonpf[k] * rpi / 180.0
        rcosphi = math.cos(rlat)
        rnomin += rcosphi * (rlabda - rlon) ** 2
    
    repsilon = rnomin / (2.0 * knpf)
    rphimean = rphisum / float(knpf)
    rphi = rphimean + repsilon * math.sin(rphimean)
    rlatmean = rphi * 180.0 / rpi
    
    return rlatmean, rlonmean

In [62]:
def create_storms_df():
    # Load cyclone dataframe with Mean sea level pressure value
    df_storms = pdbufr.read_bufr('track_data/tc_test_track_data.bufr',
        columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
                 "pressureReducedToMeanSeaLevel"))
    # Load cyclone dataframe with Wind speed at 10m value
    df1 = pdbufr.read_bufr('track_data/tc_test_track_data.bufr',
        columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
                 "windSpeedAt10M"))
    # Add the Wind speed at 10m column to the storms dataframe 
    df_storms["windSpeedAt10M"] = df1.windSpeedAt10M
    # Storms with number higher than 10 are not real storms (according to what Fernando said)
    drop_condition = df_storms.stormIdentifier < '11'
    df_storms = df_storms[drop_condition]
    return df_storms

df_storms = create_storms_df()
df_storms.head()

,stormIdentifier,ensembleMemberNumber,latitude,longitude,pressureReducedToMeanSeaLevel,windSpeedAt10M
0,07E,1,16.6,-128.9,99900.0,18.5
1,07E,1,16.3,-130.1,100200.0,18.0
2,07E,1,16.4,-131.3,100200.0,17.0
3,07E,1,16.3,-132.4,100500.0,16.0
4,07E,1,16.4,-133.2,100300.0,14.4


In [77]:
# Function that returns the list of coordinates for the mean forecast track
def mean_forecast_track(df_storm):
    
    # Create 2 dataframe with latitude and logitude coordinates for each ensemble as columns
    members = df_storm.ensembleMemberNumber.unique()
    df_lat_tracks = pd.DataFrame()
    df_lon_tracks = pd.DataFrame()
    for member in members:
        df_track = df_storm[df_storm.ensembleMemberNumber == member]
        df_track.reset_index(inplace=True)
        df_lat_tracks[f'latitude{member}'] = df_track.latitude
        df_lon_tracks[f'longitude{member}'] = df_track.longitude
    
    # Cycle through the rows of df_lat_track and df_lon_tracks to compute the average track lat,lon
    mean_track_coord = []
    for t in range(len(df_lat_tracks)):
        lat = df_lat_tracks.iloc[t].dropna().to_numpy()
        lon = df_lon_tracks.iloc[t].dropna().to_numpy()
        if len(lat) > 0:
            mean_lat_lon = meanposit(len(lat), lat, lon)
            mean_track_coord.append(mean_lat_lon)
        
    return mean_track_coord

df_storm = df_storms[df_storms.stormIdentifier == '07E']
mean_forecast_track(df_storm)

[(16.669236916133173, -128.7307712284718),
 (16.451939590539663, -130.0019229924616),
 (16.380794722815263, -131.33268038956112),
 (16.34404991891486, -132.621989614373),
 (16.37653577286502, -133.75683884771323),
 (16.325099194937756, -134.92304439509545),
 (16.33738492713745, -136.252899707797),
 (16.38619757924068, -137.481939643492),
 (16.474121131457434, -138.64992316848307),
 (16.48801040704044, -139.8672182099242),
 (16.521226552564972, -141.29352353979138),
 (16.51358564785938, -142.54952269813367),
 (16.716157043465504, -143.73738955988244),
 (16.700774199144938, -144.95857599510558),
 (16.74580140238218, -146.23201738807526),
 (16.72950154052681, -147.3565561417128),
 (16.62843444885739, -148.46302510992297),
 (16.79635213789715, -150.03754240855667),
 (16.567974602539774, -150.86659174728152),
 (15.859384130037599, -152.07150941575483),
 (16.050603170078674, -155.00067668851926),
 (14.503146323399715, -154.898401341132),
 (14.951442881493653, -156.62457058896743),
 (14.40312

In [81]:
df_storm[df_storm.ensembleMemberNumber == 2]

,stormIdentifier,ensembleMemberNumber,latitude,longitude,pressureReducedToMeanSeaLevel,windSpeedAt10M
29,07E,2,16.7,-128.5,100000.0,18.0
30,07E,2,16.4,-129.7,100300.0,17.5
31,07E,2,16.4,-131.0,100200.0,16.0
32,07E,2,16.3,-132.3,100500.0,16.5
33,07E,2,16.3,-133.4,100300.0,15.4
34,07E,2,16.2,-134.6,100600.0,14.9
35,07E,2,15.9,-136.0,100400.0,13.9
36,07E,2,16.1,-137.2,100700.0,14.4
37,07E,2,16.2,-138.0,100500.0,13.9
38,07E,2,16.1,-139.1,100800.0,14.9
